In [1]:
import pandas as pd

In [2]:
base_dir = './data/'

data = pd.read_csv(base_dir + 'custom_train_2.csv')
data.head()

,idx,class,conversation,label
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...,0
1,0,협박 대화,. 죄송합니다 . 정말 잘못했습니다 . 너에게는 선택권이 없어 . 선택 못한다면 너...,0
2,1,협박 대화,길동경찰서입니다 . 9시 40분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 ...,0
3,1,협박 대화,. 장난전화는 업무방해죄에 해당됩니다 . 판단은 너에게 달려있다 . 길동경찰서에도...,0
4,2,기타 괴롭힘 대화,너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 ...,3


# fasttext txt준비

In [3]:
def add_label(x):
    label = x['label']
    return f'''__label__{label}    {x['conversation']}'''
    

add_label_text = data.apply(add_label, axis=1)
add_label_text

0       __label__0    지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 ...
1       __label__0    . 죄송합니다 . 정말 잘못했습니다 . 너에게는 선택권이 ...
2       __label__0    길동경찰서입니다 . 9시 40분 마트에 폭발물을 설치할거다...
3       __label__0     . 장난전화는 업무방해죄에 해당됩니다 . 판단은 너에게 ...
4       __label__3    너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . ...
                              ...                        
9895    __label__4    얘들아 비트코인 일억 간다는 소리가 있던데 지금 풀매수 할...
9896    __label__4    엄마 코로나 백신 2차까지 다 맞았다고 했지 ? 엄마는 다...
9897    __label__4    가을이라 식욕 돋아서 먹방 여행 가고 싶어요저는 예전에 죽...
9898    __label__4    아니 우주 여행 가는 거 아십니까 ? 우주 여행 돈이 집 ...
9899    __label__4    나는 그때 손목 아파서 병원 오래 다녔어허허이 . . . ...
Length: 9900, dtype: object

In [4]:
# fasttext 입력으로 주려고 txt 파일 준비
file_path = 'input_fasttext_conversation.txt'

with open(file_path, 'w', encoding='utf8') as f:
    f.write('\n'.join(add_label_text))

# Model

In [5]:
!pip install wandb==0.16.0

In [6]:
import wandb

# wandb.login(key = 'YOUR KEY')

wandb: Currently logged in as: dev-dc-hyeon (devdchyeon). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [7]:
#                                       epoch=100,
#                                       bucket = 20000,
#                                       lr = 1,
#                                       wordNgrams=2,
#                                       dim=100,


# 하이퍼파라미터 검색 공간 정의
sweep_config = {
    "name": "fasttext_sweep_4",
    'method': 'bayes',  # Random search 방식으로 하이퍼파라미터 탐색
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'lr': {
            'values': [0.01, 0.05, 0.1, 0.2, 0.5]  # 더 넓은 범위의 학습률
        },
        'epoch': {
            'values': [50, 70, 100]  # 다양한 에포크 수
        },
        'wordNgrams': {
            'values': [2, 3, 4]  # 다양한 n-그램 값
        },
        'dim': {
            'values': [100, 150, 200]  # 다양한 벡터 차원
        },
        'minCount': {
            'values': [1, 2, 3, 5, 7]  # 다양한 최소 단어 출현 빈도
        }
    }
}


In [8]:
# validation 각라벨별 200개씩

val_0 = data[data['label']== 0][:200]
val_1 = data[data['label']== 1][:200]
val_2 = data[data['label']== 2][:200]
val_3 = data[data['label']== 3][:200]
val_4 = data[data['label']== 4][:200]

val_total = pd.concat([val_0, val_1, val_2, val_3, val_4])

In [9]:
import fasttext
from sklearn.metrics import accuracy_score

def train():

    
    
    
    default_config = {
          "epoch":100,
          "bucket" : 20000,
          "lr" : 1,
          "wordNgrams":2,
          "dim":100,
    }
    
    import os

    # 환경 변수 설정
    os.environ['WANDB_AGENT_DISABLE_FLAPPING'] = 'true'
    
    # WandB 프로젝트 설정
    wandb.init(config= sweep_config, project="DLTHON")

    config = wandb.config
    model = fasttext.train_supervised(input=file_path,
#                                       epoch=100,
#                                       bucket = 20000,
#                                       lr = 1,
#                                       wordNgrams=2,
#                                       dim=100,
                                      lr = config.lr,
                                      epoch = config.epoch,
                                      wordNgrams = config.wordNgrams,
                                      dim=config.dim,
                                      minCount=config.minCount
                                      )
    
    val_lables = ['__label__'+str(label) for label in val_total['label']]
    val_texts = [str(text) for text in val_total['conversation']]

    predictions = [model.predict(text)[0][0] for text in val_texts]
    accuracy = accuracy_score(val_lables, predictions)

    wandb.log({"accuracy": accuracy})

In [10]:
# Sweep 실행

# Sweep 생성
sweep_id = wandb.sweep(sweep_config, entity = 'devdchyeon', project="DLTHON")

wandb.agent(sweep_id, train, count=50)

Create sweep with ID: z3k1w1jj
Sweep URL: https://wandb.ai/devdchyeon/DLTHON/sweeps/z3k1w1jj


wandb: Agent Starting Run: j46wgis7 with config:
wandb: 	dim: 200
wandb: 	epoch: 70
wandb: 	lr: 0.5
wandb: 	minCount: 7
wandb: 	wordNgrams: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Read 0M words
Number of words:  5418
Number of labels: 5
Progress: 100.0% words/sec/thread:  401399 lr:  0.000000 avg.loss:  0.044358 ETA:   0h 0m 0s 67.9% words/sec/thread:  415429 lr:  0.160320 avg.loss:  0.065029 ETA:   0h 0m 7ss


accuracy,▁
accuracy,1.0


wandb: Agent Starting Run: xxtugrdu with config:
wandb: 	dim: 200
wandb: 	epoch: 70
wandb: 	lr: 0.5
wandb: 	minCount: 5
wandb: 	wordNgrams: 2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Read 0M words
Number of words:  5418
Number of labels: 5
Progress: 100.0% words/sec/thread:  383907 lr:  0.000000 avg.loss:  0.154465 ETA:   0h 0m 0s


accuracy,▁
accuracy,1.0


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: wlzz9gmo with config:
wandb: 	dim: 100
wandb: 	epoch: 100
wandb: 	lr: 0.2
wandb: 	minCount: 7
wandb: 	wordNgrams: 4
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Read 0M words
Number of words:  5418
Number of labels: 5
Progress: 100.0% words/sec/thread:  509176 lr:  0.000000 avg.loss:  0.069464 ETA:   0h 0m 0s  0.1% words/sec/thread:   39109 lr:  0.199835 avg.loss:  1.613349 ETA:   0h 6m 4s


accuracy,▁
accuracy,1.0


# test data

In [ ]:
# {
#     '협박 대화': 0,
#     '갈취 대화': 1,
#     '직장 내 괴롭힘 대화': 2,
#     '기타 괴롭힘 대화': 3,
#     '일반 대화': 4
# }

In [ ]:
test = pd.read_csv('./data/custom_test.csv')

test['conversation'] = test['conversation'].apply(preprocess_sentence)
test.head()

In [ ]:
predictions = model.predict(list(test['conversation']))

# return
# (labels, probabilities)

len(predictions[0])


In [ ]:
from collections import Counter

results = []
for pred in predictions[0]:
    pred = pred[0].replace('__label__','')
    results.append(pred)

# print(results)
Counter(results)

In [ ]:
# 'label' 값을 기준으로 'class' 컬럼 추가
class_dict = {
    0: '협박 대화',
    1: '갈취 대화',
    2: '직장 내 괴롭힘 대화',
    3: '기타 괴롭힘 대화',
    4: '일반 대화'
}

In [ ]:
label_df = pd.DataFrame(columns=['label'])
test['label'] = results
test['label'] = test['label'].astype(int)
test['class'] = test['label'].map(class_dict)

test.head()


In [ ]:
samples= test.sample(100)

for text,class_name in zip(samples['conversation'],samples['class']):
    print('-'*30)
    print(text)
    print(class_name)

# submission

In [ ]:
submission = pd.read_csv('./data/submission.csv')
submission['class'] = test['label']

In [ ]:
submission.head()

In [ ]:
submission.to_csv('my_submission.csv', index=False)

In [ ]:

text = ('장애인이라고 너무 무시하시는거 아닌가요 ? 무시라니요 . 못 따라 오는거 같아보이던데 . 지혜는 따라가려고 노력한다는데 선생님께서 기회를 안주신다고 해서요 . 무슨말이에요 . 지혜가 그래요 ? 수업시간에 활동할 때에도 친구들 도와주는 보조역할만 시키신다고 그래서요 . 네 ? 그거야 지헤는 못하잖아요 . 할줄 아는게 뭐가 있어요 . 선생님이 도와주시면 충분히 지혜도 또래들과 어울릴 수 있는데 . 아니요 . 지혜가 어떻게 일반 아이들과 따라가요 . 따라갈 수도 있죠 . 지혜가 학급에 있는건 다른 친구들에게 모욕이에요 .')

labels, probabilities = model.predict(text, k=-1)  # k=-1이면 모든 클래스에 대한 확률을 반환

print(f"Text: {text}")
for label, probability in zip(labels, probabilities):
    print(f"  {label}: {probability:.4f}")


NameError: name 'test' is not defined